# Topic Modeling
### Without Preprocessing

In [1]:
import json
import os
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
import spacy
# import coreferee
# nlp = spacy.load('en_core_web_trf')
# nlp.add_pipe('coreferee')
nlp = spacy.load("en_core_web_sm")

2022-08-28 17:59:26.215100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-28 17:59:26.215146: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Define Parameters

In [2]:
n_components = 5 #Topics To generate
n = 40 #Transcripts to use
max_df=20
min_df=3
n_terms = 10

## Load Data

In [3]:
# pos = ["CCONJ","NUM","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
# pos = ["ADJ","ADP","ADV","AUX","CCONJ","DET","INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","VERB","X"]
pos = ["ADJ","AUX","CCONJ","DET","INTJ","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
def clean_text(text, coref = False, lemma = True, pos_exclusions = pos):
    doc = nlp(text)

    if coref:
        new_text = ""
        for i  in range(len(doc)):
            tokens = doc[i]
            if doc._.coref_chains.resolve(tokens):
                new_text += doc._.coref_chains.resolve(tokens)[0].text
            else:
                new_text += tokens.text
            new_text+= " "
        doc = nlp(new_text)
    text = ""
    for token in doc:
        if token.pos_ not in pos_exclusions:
            text += token.lemma_ if lemma else token.text
            text += " "
    return text

In [4]:
def create_df_multi(n=25):
    list_of_text = []
    dir = 'data/aligned data/c=4'

    files = [filename for filename in os.listdir(dir)]
    sampled_files = random.choices(files,k=n)

    for filename in sampled_files:
        # choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+filename)
        data = json.load(f)
        choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+choice)
        data = json.load(f)

        for x in data:
            for y in x['TURNS']:
                text = ' '.join(y['UTTERANCES'])

                text = clean_text(text)
                list_of_text.append(text)
    df = pd.DataFrame(list_of_text)
    return df

In [5]:
df = create_df_multi(n)
print(df.shape)
df.head()

(105048, 1)


,0
0,welcome evening to show bunch of ass voice act...
1,welcome back so first foremost get couple anno...
2,so
3,slightly
4,go worry about want apologize in advance to fo...


## Vectorize Data



- Count Vectorizer
- Tfidf Vectorizer

Use both with each model

In [6]:
# tfidf=TfidfVectorizer(stop_words='english',max_df=.7,min_df=2,token_pattern=r'(?u)\b[A-Za-z]+\b')

tfidf = TfidfVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
tfidf_sparse = tfidf.fit_transform(df[0])
print(tfidf_sparse.shape)
tfidf_df = pd.DataFrame(tfidf_sparse.toarray().transpose(),
                   index=tfidf.get_feature_names_out())
tfidf_df.tail()

(105048, 4191)


,0,1,2,3,4,5,6,7,8,9,...,105038,105039,105040,105041,105042,105043,105044,105045,105046,105047
zigzag,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zip,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zoo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
cv = CountVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
cv_sparse = cv.fit_transform(df[0])
print(cv_sparse.shape)
cv_df = pd.DataFrame(cv_sparse.toarray().transpose(),
                   index=cv.get_feature_names_out())
cv_df.tail()

(105048, 4191)


,0,1,2,3,4,5,6,7,8,9,...,105038,105039,105040,105041,105042,105043,105044,105045,105046,105047
zigzag,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zombie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Define Output Functions

In [19]:
def print_top_terms(n_components, topics, terms, n_terms = 10):
    for x in range(n_components):
        topic = x
        components = topics[:,topic]
        top_term_indices = components.argsort()[-n_terms:]
        top_terms = np.array(terms)[top_term_indices]
        
        print(f'Topic {x}:\t{top_terms.tolist()}')


## PCA

In [17]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['contemplate', 'cling', 'creator', 'ceremony', 'looter']
Topic 1:	['breeze', 'drape', 'port', 'ceremony', 'looter']
Topic 2:	['eternity', 'completion', 'obligation', 'scribe', 'recipient']
Topic 3:	['meditate', 'truly', 'childhood', 'impression', 'headmaster']
Topic 4:	['reign', 'demand', 'devour', 'impression', 'headmaster']


In [18]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['tick', 'pen', 'toast', 'theater', 'eyeball']
Topic 1:	['lately', 'advisor', 'tearfully', 'locket', 'grieve']
Topic 2:	['marching', 'vomit', 'gingerly', 'theater', 'locket']
Topic 3:	['rpg', 'lycan', 'locket', 'vomit', 'invisibility']
Topic 4:	['warn', 'grieve', 'gingerly', 'theater', 'invisibility']


## LDA

In [15]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['mimic', 'site', 'gnoll', 'chant', 'artist']
Topic 1:	['encampment', 'advisor', 'looter', 'hunter', 'impression']
Topic 2:	['membership', 'caravan', 'orchard', 'pew', 'fortress']
Topic 3:	['invisibility', 'lemure', 'thousand', 'account', 'mini']
Topic 4:	['vary', 'ravine', 'shackle', 'tablet', 'tick']


In [16]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['sizzle', 'toast', 'queen', 'lycan', 'theater']
Topic 1:	['disarm', 'gone', 'sure', 'yesterday', 'grieve']
Topic 2:	['lately', 'advisor', 'tary', 'vomit', 'invisibility']
Topic 3:	['tablet', 'hex', 'tick', 'gingerly', 'eyeball']
Topic 4:	['tearfully', 'mimic', 'text', 'locket', 'pen']


## SVD

In [13]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['contemplate', 'cling', 'creator', 'ceremony', 'looter']
Topic 1:	['contemplate', 'cling', 'creator', 'ceremony', 'looter']
Topic 2:	['eternity', 'completion', 'obligation', 'scribe', 'recipient']
Topic 3:	['childhood', 'meditate', 'truly', 'impression', 'headmaster']
Topic 4:	['reign', 'demand', 'devour', 'impression', 'headmaster']


In [14]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['warhammer', 'sizzle', 'locket', 'gingerly', 'eyeball']
Topic 1:	['lately', 'yesterday', 'lycan', 'locket', 'grieve']
Topic 2:	['toast', 'lycan', 'pen', 'vomit', 'gingerly']
Topic 3:	['toast', 'gingerly', 'mimic', 'locket', 'pen']
Topic 4:	['goes', 'locket', 'yesterday', 'theater', 'tearfully']


## NMF

In [10]:
nmf = NMF(n_components=n_components, max_iter=500)
topics = nmf.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['ware', 'sage', 'drape', 'breeze', 'port']
Topic 1:	['contemplate', 'cling', 'creator', 'ceremony', 'looter']
Topic 2:	['eternity', 'completion', 'obligation', 'scribe', 'recipient']
Topic 3:	['meditate', 'childhood', 'truly', 'impression', 'headmaster']
Topic 4:	['orcs', 'puzzle', 'reign', 'demand', 'devour']


In [11]:
nmf = NMF(n_components=n_components, max_iter=400)
topics = nmf.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

/home/patrick/anaconda3/envs/metis/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


Topic 0:	['coil', 'louder', 'relief', 'socket', 'eyeball']
Topic 1:	['permission', 'approximately', 'jog', 'lycan', 'grieve']
Topic 2:	['apparatus', 'bore', 'slice', 'disarm', 'gingerly']
Topic 3:	['latch', 'wool', 'backpack', 'tally', 'locket']
Topic 4:	['warn', 'stork', 'represent', 'invisibility', 'theater']
